<h2>Question 5</h2>

<h3>Loading data</h3>

In [27]:
import pandas as pd
df_votes = pd.read_csv('votes.csv')
print("Completed loading votes.csv")
df_post = pd.read_csv('posts.csv')
print("Completed loading post.csv")

Completed loading votes.csv
Completed loading post.csv


<h3>Ignoring the red warnings when processing through the data. Not necessary to run the program, but it will give neat output.</h3>

In [28]:
import warnings
warnings.filterwarnings('ignore')

<h3>Main Function responsible for calculating the reputation</h3>

In [31]:
def reput(user_id,date_limit):
    df_post_user = df_post[df_post['owner_user_id'] == user_id]
    df_post_user['creation_date'] =  pd.to_datetime(df_post_user['creation_date'], format='%Y-%m-%dT%H:%M:%S.%f')
    df_post_user = df_post_user[df_post_user['creation_date'] < date_limit]
    df_post_user['time_since'] = (df_post_user['creation_date'] - df_post_user.iloc[0].creation_date ).dt.days
    #Currently df_post_user has posts which have owner_user_id= user_id and creation time less then the date supplied
    
    total_points=0  #Storing the points
    days_checked=[] #Storing days which are processed to avoid calculating same day repeatedly
    
    #Number of posts to processing having the required user id and creation date
    #print("Length of df_post_user:",len(df_post_user))
    for i in range(len(df_post_user)):
        row = df_post_user.iloc[i]
        day_to_be_checked=row.time_since #Day which is to be processed
        if day_to_be_checked not in days_checked: #Checking if the same day has been processed earlier or not. Like user can have multiple posts on same day resulting in time since being same for all these multiple post, while each having unique row.
            temp_df=df_post_user[df_post_user['time_since'] == day_to_be_checked] #Temporary dataframe containing posts with the same date as required
            day_points=0 # Varaible for storing daily reputation gain
            for j in range(len(temp_df)): #iterating throught the temporary dataframe and calculating the reputation
                row_temp = temp_df.iloc[j]
                #temp_df_2 = df_votes[(df_votes['post_id'] == row_temp.id) and (df_votes['vote_type_id'] == 2)]
                temp_df_2 = df_votes[(df_votes['post_id'] == row_temp.id)]
                temp_df_2 = temp_df_2[(temp_df_2['vote_type_id'] == 2)] #Resulting temp_df_2 has number of rows equal to number of upvotes, it doesn't matter if its upvoted question or answer as both have same effect on reputation
                for k in range(len(temp_df_2)):
                    day_points=day_points+10 #Increasing daily reputation gain varialbe by 10 for upvotes

                #temp_df_2 = df_votes[(df_votes['post_id'] == row_temp.id) and (df_votes['vote_type_id'] == 3)]
                temp_df_2 = df_votes[(df_votes['post_id'] == row_temp.id)]
                temp_df_2 = temp_df_2[(temp_df_2['vote_type_id'] == 3)] ##Resulting temp_df_2 has number of rows equal to number of downvotes, it doesn't matter if its downvote question or answer as both have same effect on reputation
                for k in range(len(temp_df_2)):
                    day_points=day_points-2 #Decreasing daily reputation gain variable by 2 for downvotes
            if(day_points>200): # Limiting daily reputation gain to 200
                day_points=200
            total_points=total_points+day_points
            days_checked.append(day_to_be_checked)
        else:
            pass
        
        #Calculating the total accepted answers given by user, its outside daily calculation as it doesn't have daily limit
        if((df_post['accepted_answer_id'] == row.id)).any():
#             print("Got accepted answer")
            total_points=total_points+15
    #     print(total_points)
    return total_points


<h3>Getting inputs from user and printing the reputation as required</h3>

In [32]:
user=int(input("Enter the user ID:"))
date=(input("Enter the date in proper format in proper format eg. 2010-02-19 ie.YYYY-MM-DD "))
# print(reput(9,"2010-02-19"))
print("Reputation of entered user is:",reput(user,date))

Enter the user ID:9
Enter the date in proper format in proper format eg. 2010-02-19 ie.YYYY-MM-DD 2010-02-19
Reputation of entered user is: 1678
